## Day 16 - light beaming about

Light hitting travelling LEFT to RIGHT:
 - `-` means continue
 - `.` means continue
 - `\` means reflected down
 - `/` means reflected up
 - `|` means split into two, up and down

**Part 1: How many tiles do beams of light pass over?**

In [234]:
with open("./example.txt") as f:
    examples_lines = [line.strip() for line in f.readlines()]

with open("./input.txt") as f:
    input_lines = [line.strip() for line in f.readlines()]

examples_lines

['.|...\\....',
 '|.-.\\.....',
 '.....|-...',
 '........|.',
 '..........',
 '.........\\',
 '..../.\\\\..',
 '.-.-/..|..',
 '.|....-|.\\',
 '..//.|....']

Let's steal some code from Day 10...

In [236]:
class Vector2D():
    def __init__(self, y, x):
        """
        np grid works like 
        (y,x)    (y, x+1)    (y, x+2)
        (y+1, x) (y+1, x+1)  (y+1, x+2)
        (y+2, x) .......
        """
        self.y = y
        self.x = x

    def __add__(self, v2):
        return Vector2D(self.y + v2.y, self.x + v2.x)
    
    def __sub__(self, v2):
        return Vector2D(self.y - v2.y, self.x - v2.x)

    def __iadd__(self, v2):
        self.x += v2.x
        self.y += v2.y
        return self
    
    def __isub__(self, v2):
        self.x -= v2.x
        self.y -= v2.y
        return self
    
    def __str__(self):
        return f'({self.y},{self.x})'
    
    def __eq__(self, v2):
        return (self.x==v2.x and self.y==v2.y)

    @property
    def as_tuple(self):
        return (self.y, self.x)

In [237]:
def create_grid_map(lines: list[str]) -> dict:
    """
    Takes in the input text (separated by line into a list)
    and notes the value at each location in a dict

    so key in like position (1,2) and get value at that location.

    USING SYSTEM:
        (y, x)
        where (0,0) is top left corner

    Returns dict map and also the starting position!
    """
    grid_map = {}
    for y_index, line in enumerate(lines):
        for x_index, val in enumerate(line):
            grid_map[(y_index, x_index)] = val

    return grid_map

In [238]:
from enum import Enum

class LightBeamDirections(Enum):
    UP = Vector2D(-1,0)
    DOWN = Vector2D(1,0)
    LEFT = Vector2D(0,-1)
    RIGHT = Vector2D(0,1)

In [239]:
def get_new_direction_from_instruction_and_relative_pos(
        instruction: str, relative_pos: Vector2D
    ) -> Vector2D | tuple[Vector2D, Vector2D]:
    """
    relative_position means instruction_pos - current_pos

    Instructions: - | / \\ .

    Returns:
        - new_direction : Vector2D
        OR tuple(new_direction, new_direction), both vecs
    """
    if relative_pos == LightBeamDirections.RIGHT.value:
        if instruction in ".-":
            return LightBeamDirections.RIGHT.value
        if instruction == "\\":
            return LightBeamDirections.DOWN.value
        if instruction == "/":
            return LightBeamDirections.UP.value
        if instruction ==  "|":
            return (LightBeamDirections.UP.value, LightBeamDirections.DOWN.value)
    
    if relative_pos == LightBeamDirections.LEFT.value:
        if instruction in ".-":
            return LightBeamDirections.LEFT.value
        if instruction == "\\":
            return LightBeamDirections.UP.value
        if instruction == "/":
            return LightBeamDirections.DOWN.value
        if instruction ==  "|":
            return (LightBeamDirections.UP.value, LightBeamDirections.DOWN.value)
    
    if relative_pos == LightBeamDirections.UP.value:
        if instruction in ".|":
            return LightBeamDirections.UP.value
        if instruction == "\\":
            return LightBeamDirections.LEFT.value
        if instruction == "/":
            return LightBeamDirections.RIGHT.value
        if instruction ==  "-":
            return (LightBeamDirections.LEFT.value, LightBeamDirections.RIGHT.value)
    
    if relative_pos == LightBeamDirections.DOWN.value:
        if instruction in ".|":
            return LightBeamDirections.DOWN.value
        if instruction == "\\":
            return LightBeamDirections.RIGHT.value
        if instruction == "/":
            return LightBeamDirections.LEFT.value
        if instruction ==  "-":
            return (LightBeamDirections.LEFT.value, LightBeamDirections.RIGHT.value)
    
    raise ValueError(f"What are we doing with relative position {relative_pos} and instruction {instruction} here?")


In [240]:
import copy

class LightBeam():

    def __init__(self, starting_position: Vector2D = Vector2D(0,0), current_dir: Vector2D = Vector2D(0,1)):
        starting_position = copy.deepcopy(starting_position)
        self.pos: Vector2D = copy.deepcopy(starting_position)
        self.current_dir: Vector2D = copy.deepcopy(current_dir)
        self.dead = False
    
    def move(self, dir : Vector2D = None):
        """
        moves the light beam in chosen direction.
        """

        # manually

        # Dont go backwards surely
        if self.current_dir == LightBeamDirections.RIGHT.value and dir == LightBeamDirections.LEFT.value:
            raise ValueError("Backwards!!!")
        if self.current_dir == LightBeamDirections.LEFT.value and dir == LightBeamDirections.RIGHT.value:
            raise ValueError("Backwards!!!")
        if self.current_dir == LightBeamDirections.UP.value and dir == LightBeamDirections.DOWN.value:
            raise ValueError("Backwards!!!")
        if self.current_dir == LightBeamDirections.DOWN.value and dir == LightBeamDirections.UP.value:
            raise ValueError("Backwards!!!")

        # Set new current direction
        self.current_dir = dir
        # Update position
        self.pos += self.current_dir

In [241]:
class LightBeams():
    def __init__(self, grid_map: dict, light_beams: list[LightBeam], max_y: int, max_x: int):
        self.grid_map = grid_map
        self.light_beams = light_beams
        self.seen_locs_with_dirs = {
            (lb.pos.as_tuple, lb.current_dir.as_tuple)
            for lb in light_beams
        }
        self.active = True

        self.max_y = max_y
        self.max_x = max_x

        self.next_batch_of_beams = []

    
    def move_all(self):
        for light_beam in self.light_beams:
            relative_position_to_next_point = light_beam.current_dir
            try:
                next_instruction = self.grid_map[(light_beam.pos + light_beam.current_dir).as_tuple]
                next_direction = get_new_direction_from_instruction_and_relative_pos(
                    next_instruction, relative_position_to_next_point
                )

                if isinstance(next_direction, tuple):
                    # Split case!
                    next_dir1, next_dir2 = next_direction
                    # original
                    light_beam.pos += light_beam.current_dir
                    light_beam.current_dir = next_dir1

                    # Spawned
                    new_light_beam = LightBeam(starting_position=light_beam.pos, current_dir=next_dir2)

                    if (new_light_beam.pos.as_tuple, new_light_beam.current_dir.as_tuple) not in self.seen_locs_with_dirs:
                        self.next_batch_of_beams.append(copy.deepcopy(new_light_beam))
                        # else there's not much point spawning it if we've had it before!

                else:
                    light_beam.pos += light_beam.current_dir
                    light_beam.current_dir = next_direction
                
                if (light_beam.pos.as_tuple, light_beam.current_dir.as_tuple) in self.seen_locs_with_dirs:
                    light_beam.dead = True

            except KeyError:
                # means we're off the grid!
                light_beam.dead = True
    
    def declare_dead(self):
        for light_beam in self.light_beams:
            y_position, x_position = light_beam.pos.as_tuple
            if y_position < 0 or y_position > self.max_y:
                light_beam.dead = True
            elif x_position < 0 or x_position > self.max_x:
                light_beam.dead = True
    
    def play_once(self):
        self.move_all()
        self.declare_dead()

        if all(lb.dead for lb in self.light_beams):
            if self.next_batch_of_beams:
                self.light_beams = copy.deepcopy(self.next_batch_of_beams)
                self.next_batch_of_beams = []
            else:
                self.active = False

        new_potential_locs_with_dirs = set()
        for light_beam in self.light_beams:
            if not light_beam.dead:
                tmp_potential_loc_with_dir = (
                    light_beam.pos.as_tuple, light_beam.current_dir.as_tuple
                )
                new_potential_locs_with_dirs |= {tmp_potential_loc_with_dir}
        
        self.seen_locs_with_dirs |= new_potential_locs_with_dirs
    
    @property
    def num_of_unique_seen_locs(self):
        unique_locs = set(loc for loc, _ in self.seen_locs_with_dirs)
        return len(unique_locs)


In [242]:
def part1(lines: list[str]) -> int:
    max_y = len(lines) - 1
    max_x = len(lines[0]) - 1
    grid_map = create_grid_map(lines)
    initial_direction = get_new_direction_from_instruction_and_relative_pos(
        grid_map[(0,0)], LightBeamDirections.RIGHT.value
    )
    initial_light_beam = LightBeam(current_dir=initial_direction)

    light_beams = LightBeams(grid_map, [initial_light_beam], max_y, max_x)

    while light_beams.active:
        light_beams.play_once()
    
    return light_beams.num_of_unique_seen_locs

assert part1(examples_lines) == 46
part1(input_lines)

7067

**Part 2 - Find starting position that energises the most from all possible starting points!**

In [253]:
def part2(lines: list[str]) -> int:
    max_y = len(lines) - 1
    max_x = len(lines[0]) - 1
    grid_map = create_grid_map(lines)

    initial_beams = []
    for y in range(max_y+1):
        for x in range(max_x+1):
            if y in (0, max_y) or x in (0, max_x):
                starting_pos = Vector2D(y, x)

                # do corners first then general cases
                if y == 0 and x == 0:
                    # DOWN
                    starting_dir = get_new_direction_from_instruction_and_relative_pos(
                        grid_map[starting_pos.as_tuple], LightBeamDirections.DOWN.value
                    )
                    if isinstance(starting_dir, tuple):
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir[1])
                        )
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir[0])
                        )
                    else:
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir)
                        )
                    # RIGHT
                    starting_dir = get_new_direction_from_instruction_and_relative_pos(
                        grid_map[starting_pos.as_tuple], LightBeamDirections.RIGHT.value
                    )
                    if isinstance(starting_dir, tuple):
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir[1])
                        )
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir[0])
                        )
                    else:
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir)
                        )
                    
                    continue
                
                if y == 0 and x == max_x:
                    # DOWN
                    starting_dir = get_new_direction_from_instruction_and_relative_pos(
                        grid_map[starting_pos.as_tuple], LightBeamDirections.DOWN.value
                    )
                    if isinstance(starting_dir, tuple):
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir[1])
                        )
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir[0])
                        )
                    else:
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir)
                        )
                    # RIGHT
                    starting_dir = get_new_direction_from_instruction_and_relative_pos(
                        grid_map[starting_pos.as_tuple], LightBeamDirections.LEFT.value
                    )
                    if isinstance(starting_dir, tuple):
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir[1])
                        )
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir[0])
                        )
                    else:
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir)
                        )
                    
                    continue
            
                if y == max_y and x == 0:
                    # RIGHT
                    starting_dir = get_new_direction_from_instruction_and_relative_pos(
                        grid_map[starting_pos.as_tuple], LightBeamDirections.RIGHT.value
                    )
                    if isinstance(starting_dir, tuple):
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir[1])
                        )
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir[0])
                        )
                    else:
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir)
                        )
                    # UP
                    starting_dir = get_new_direction_from_instruction_and_relative_pos(
                        grid_map[starting_pos.as_tuple], LightBeamDirections.UP.value
                    )
                    if isinstance(starting_dir, tuple):
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir[1])
                        )
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir[0])
                        )
                    else:
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir)
                        )
                    
                    continue
                
                if y == max_y and x == max_x:
                    # LEFT
                    starting_dir = get_new_direction_from_instruction_and_relative_pos(
                        grid_map[starting_pos.as_tuple], LightBeamDirections.LEFT.value
                    )
                    if isinstance(starting_dir, tuple):
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir[1])
                        )
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir[0])
                        )
                    else:
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir)
                        )
                    # UP
                    starting_dir = get_new_direction_from_instruction_and_relative_pos(
                        grid_map[starting_pos.as_tuple], LightBeamDirections.UP.value
                    )
                    if isinstance(starting_dir, tuple):
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir[1])
                        )
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir[0])
                        )
                    else:
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir)
                        )
                    
                    continue

                if y == 0:
                    # on the top
                    starting_dir = get_new_direction_from_instruction_and_relative_pos(
                        grid_map[starting_pos.as_tuple], LightBeamDirections.DOWN.value
                    )
                    if isinstance(starting_dir, tuple):
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir[1])
                        )
                        starting_dir = starting_dir[0]
                elif y == max_x:
                    starting_dir = get_new_direction_from_instruction_and_relative_pos(
                        grid_map[starting_pos.as_tuple], LightBeamDirections.UP.value
                    )
                    if isinstance(starting_dir, tuple):
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir[1])
                        )
                        starting_dir = starting_dir[0]
                elif x == 0:
                    starting_dir = get_new_direction_from_instruction_and_relative_pos(
                        grid_map[starting_pos.as_tuple], LightBeamDirections.RIGHT.value
                    )
                    if isinstance(starting_dir, tuple):
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir[1])
                        )
                        starting_dir = starting_dir[0]
                elif x == max_x:
                    starting_dir = get_new_direction_from_instruction_and_relative_pos(
                        grid_map[starting_pos.as_tuple], LightBeamDirections.LEFT.value
                    )
                    if isinstance(starting_dir, tuple):
                        initial_beams.append(
                            LightBeam(starting_pos, starting_dir[1])
                        )
                        starting_dir = starting_dir[0]

                initial_beams.append(
                    LightBeam(starting_pos, starting_dir)
                )

    max_found = 0
    for initial_beam in initial_beams:
        light_beams = LightBeams(grid_map, [initial_beam], max_y, max_x)

        while light_beams.active:
            light_beams.play_once()

        max_found = max(max_found, light_beams.num_of_unique_seen_locs)

    return max_found

assert part2(examples_lines) == 51
part2(input_lines)  # brute force go brrrr

7324